In [1]:
import requests

Creating a data structure to store the national park information in:

In [3]:
#creating a tree structure to later input national park information into:
class Node:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def __str__(self, level=0):
        ret = " " * level + self.name + "\n"
        for child in self.children:
            ret += child.__str__(level + 1)
        return ret

    def __repr__(self):
        return self.__str__()

Using the tree data structure to start creating children:

In [4]:
#create a root node called "National Parks in California"
root = Node("National Parks in California")

#creating a node for each national park in California
yosemite = Node("Yosemite", root)
sequoia = Node("Sequoia", root)
kings_canyon = Node("Kings Canyon", root)
pinnacles = Node("Pinnacles", root)
channel_islands = Node("Channel Islands", root)
lassen = Node("Lassen", root)
redwood = Node("Redwood", root)
joshua_tree = Node("Joshua Tree", root)
death_valley = Node("Death Valley", root)


Adding the child nodes to our tree

In [5]:
root.add_child(yosemite)
root.add_child(sequoia)
root.add_child(kings_canyon)
root.add_child(pinnacles)
root.add_child(channel_islands)
root.add_child(lassen)
root.add_child(redwood)
root.add_child(joshua_tree)
root.add_child(death_valley)
